In [47]:
import csv
import psycopg2

In [55]:
db_conn = None
def get_db_conn():
    global db_conn
    if db_conn is not None:
        return db_conn
    db_conn = psycopg2.connect(
        database="cogdb",
        user="sylvia",
        password="L@unchM3N0w",
        host="localhost",
        port="20000"
    )
    return db_conn


In [56]:
def ingest_parcels(csv_file, municode, pclkey_start):
    db_conn = get_db_conn()
    matchcount = 0
    reccount = 0
    maxdisp = 100
    cursor = db_conn.cursor()
    if cursor:
        print("we have a cursor!")
    else:
        print("no cursor :(")
    # Using hard-coded event type ID: 18, for code enforcement letter type
    sql_command = """
    INSERT INTO public.parcel(
            parcelkey, parcelidcnty, source_sourceid, createdts, createdby_userid, 
            lastupdatedts, lastupdatedby_userid, deactivatedts, deactivatedby_userid, 
            notes, muni_municode, lotandblock, broadview_photodocid)
    VALUES (%(SCPTKEY)s, %(PARID)s, 13 , now(), 100, 
            now(), 100, NULL, NULL, 
            'WPRDC CSV write',927, NULL, NULL);

    """
    with open(csv_file, 'r', newline='') as infile:
            reader = csv.DictReader(infile)
            for row in reader:
#                 print(row)
                if row['MUNICODE'] == municode:
                    matchcount+=1
                    row['SCPTKEY']=str(pclkey_start)
                    # Insert the data to the Postgres table
                    cursor.execute(sql_command, row)
                    db_conn.commit()    
#                     print("done inserting ", pclkey_start)
                    pclkey_start+=1
    print("matching record count: ", matchcount)

In [57]:
db_conn = get_db_conn()
ingest_parcels('/home/sylvia/cogconnect_clientdocs/mccandless/parcels.csv', '927',2000000)
# db_conn.commit()    
# db_conn.close()


we have a cursor!
matching record count:  11231
